In [1]:
from joblib import Parallel, delayed
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor
from utils import *

In [3]:
runs = load_runs()
runs

,GA,PSO,DE,CMAES,ES,algorithm_run,problem1,problem2,instance1,instance2,alpha,dim,optimum
0,4.923745,1.419528e-05,64.405318,1.000000e-20,0.095015,2,6,12,3,3,0.0,5,1.000000e-20
1,5.421025,3.278726e+01,84.558786,1.180327e-14,0.016344,2,6,12,3,3,0.1,5,1.000000e-20
2,54.710072,3.454621e+01,12.360161,1.633917e-13,0.006415,2,6,12,3,3,0.2,5,1.000000e-20
3,3.387965,3.362608e-09,2.073382,1.218820e-14,0.031416,2,6,12,3,3,0.3,5,1.000000e-20
4,349.661134,8.734929e-08,13.544310,2.289615e-15,0.107885,2,6,12,3,3,0.4,5,1.000000e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2.741841,5.406854e+00,3.881801,4.300388e-01,4.779582,2,24,19,4,4,0.6,5,1.000000e-20
7,13.100381,1.305377e+01,5.094707,1.400810e+00,7.666339,2,24,19,4,4,0.7,5,1.000000e-20
8,18.056932,3.318287e+00,10.735367,3.732188e+00,9.442364,2,24,19,4,4,0.8,5,1.000000e-20
9,18.342542,1.830129e+01,15.038300,1.021126e+01,13.510524,2,24,19,4,4,0.9,5,1.000000e-20


In [4]:
ela = load_ela()
ela

,problem1,problem2,instance1,instance2,alpha,dim,algorithm_run,disp.ratio_mean_02,disp.ratio_mean_05,disp.ratio_mean_10,...,nbc.costs_runtime,pca.expl_var.cov_x,pca.expl_var.cor_x,pca.expl_var.cov_init,pca.expl_var.cor_init,pca.expl_var_PC1.cov_x,pca.expl_var_PC1.cor_x,pca.expl_var_PC1.cov_init,pca.expl_var_PC1.cor_init,pca.costs_runtime
0,6,12,3,3,0.0,5,2,0.503606,0.608375,0.674247,...,17.748704,1.0,1.0,0.166667,0.833333,0.208432,0.208432,1.000000,0.256565,1.092185
1,6,12,3,3,0.1,5,2,0.520226,0.596088,0.683402,...,21.210949,1.0,1.0,0.166667,0.833333,0.210638,0.210639,1.000000,0.271169,1.187721
2,6,12,3,3,0.2,5,2,0.545058,0.611233,0.682499,...,19.516812,1.0,1.0,0.166667,0.833333,0.208114,0.208113,1.000000,0.273635,1.396110
3,6,12,3,3,0.3,5,2,0.518158,0.615401,0.686525,...,20.083273,1.0,1.0,0.166667,0.833333,0.205883,0.205883,1.000000,0.279927,0.755455
4,6,12,3,3,0.4,5,2,0.541886,0.608230,0.689792,...,18.917092,1.0,1.0,0.166667,0.833333,0.206389,0.206389,1.000000,0.285603,0.603599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,24,19,4,4,0.6,5,2,0.555168,0.600717,0.655388,...,19.904537,1.0,1.0,0.166667,1.000000,0.204859,0.204859,0.976757,0.204948,0.897549
7,24,19,4,4,0.7,5,2,0.554852,0.606805,0.654537,...,22.963030,1.0,1.0,0.166667,1.000000,0.204531,0.204531,0.976137,0.199709,1.090259
8,24,19,4,4,0.8,5,2,0.545253,0.605123,0.658821,...,19.894607,1.0,1.0,0.166667,1.000000,0.207689,0.207689,0.974129,0.212422,1.212369
9,24,19,4,4,0.9,5,2,0.645374,0.660286,0.686959,...,20.638718,1.0,1.0,0.166667,0.833333,0.210565,0.210565,0.977303,0.230646,2.092381


In [5]:
join_cols = ["algorithm_run", "problem1", "problem2", "instance1", "instance2", "alpha", "dim"]
result = pd.merge(ela, runs, on=join_cols, how="inner")

In [6]:
result

,problem1,problem2,instance1,instance2,alpha,dim,algorithm_run,disp.ratio_mean_02,disp.ratio_mean_05,disp.ratio_mean_10,...,pca.expl_var_PC1.cor_x,pca.expl_var_PC1.cov_init,pca.expl_var_PC1.cor_init,pca.costs_runtime,GA,PSO,DE,CMAES,ES,optimum
0,6,12,3,3,0.0,5,2,0.503606,0.608375,0.674247,...,0.208432,1.000000,0.256565,1.092185,4.923745,1.419528e-05,64.405318,1.000000e-20,0.095015,1.000000e-20
1,6,12,3,3,0.1,5,2,0.520226,0.596088,0.683402,...,0.210639,1.000000,0.271169,1.187721,5.421025,3.278726e+01,84.558786,1.180327e-14,0.016344,1.000000e-20
2,6,12,3,3,0.2,5,2,0.545058,0.611233,0.682499,...,0.208113,1.000000,0.273635,1.396110,54.710072,3.454621e+01,12.360161,1.633917e-13,0.006415,1.000000e-20
3,6,12,3,3,0.3,5,2,0.518158,0.615401,0.686525,...,0.205883,1.000000,0.279927,0.755455,3.387965,3.362608e-09,2.073382,1.218820e-14,0.031416,1.000000e-20
4,6,12,3,3,0.4,5,2,0.541886,0.608230,0.689792,...,0.206389,1.000000,0.285603,0.603599,349.661134,8.734929e-08,13.544310,2.289615e-15,0.107885,1.000000e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68844,24,19,4,4,0.6,5,2,0.555168,0.600717,0.655388,...,0.204859,0.976757,0.204948,0.897549,2.741841,5.406854e+00,3.881801,4.300388e-01,4.779582,1.000000e-20
68845,24,19,4,4,0.7,5,2,0.554852,0.606805,0.654537,...,0.204531,0.976137,0.199709,1.090259,13.100381,1.305377e+01,5.094707,1.400810e+00,7.666339,1.000000e-20
68846,24,19,4,4,0.8,5,2,0.545253,0.605123,0.658821,...,0.207689,0.974129,0.212422,1.212369,18.056932,3.318287e+00,10.735367,3.732188e+00,9.442364,1.000000e-20
68847,24,19,4,4,0.9,5,2,0.645374,0.660286,0.686959,...,0.210565,0.977303,0.230646,2.092381,18.342542,1.830129e+01,15.038300,1.021126e+01,13.510524,1.000000e-20


In [7]:
algorithms = ['GA', 'PSO', 'DE', 'CMAES', 'ES']
meta_columns = ['problem1', 'problem2', 'instance1', 'instance2', 'alpha', 'dim', 'algorithm_run']
features = [x for x in result.columns if '.' in x]

In [8]:
result = get_rank(result, algorithms)
#result = log_performances(result, algorithms)
X = result[features]
X = X.replace([np.inf, -np.inf], np.nan)
Y = result[algorithms + meta_columns]

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
Y_train_meta = Y_train[meta_columns]
Y_test_meta = Y_test[meta_columns]
Y_train = Y_train[algorithms]
Y_test = Y_test[algorithms]

In [11]:
model_xgb = XGBRegressor(n_jobs=-1, verbosity=1, eval_metric="mae")
model_xgb.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [12]:
model_dummy = DummyRegressor(strategy='median')
model_dummy.fit(X_train, Y_train)

DummyRegressor(strategy='median')

In [13]:
def normalize_rows(df, value):
    row_sum = df.sum(axis=1)
    df_norm = df.div(row_sum, axis=0).mul(value)
    return df_norm

In [14]:
#Y_test = get_rank(Y_test, algorithms)
Y_pred = pd.DataFrame(model_xgb.predict(X_test), columns=algorithms)
#Y_pred = get_rank(Y_pred, algorithms)
#Y_test = get_rank(Y_test, algorithms)

Y_pred = normalize_rows(Y_pred, value=15)
mae = mean_absolute_error(Y_test, Y_pred)
print("Mean absolute error:", mae)

Y_pred = pd.DataFrame(model_dummy.predict(X_test), columns=algorithms)
#Y_pred = get_rank(Y_pred, algorithms)
Y_pred = normalize_rows(Y_pred, value=15)
mae = mean_absolute_error(Y_test, Y_pred)
print("Mean absolute error:", mae)

Mean absolute error: 0.740994444459179
Mean absolute error: 1.0772652764809931


In [15]:
Y_pred

,GA,PSO,DE,CMAES,ES
0,4.285714,3.214286,3.214286,1.071429,3.214286
1,4.285714,3.214286,3.214286,1.071429,3.214286
2,4.285714,3.214286,3.214286,1.071429,3.214286
3,4.285714,3.214286,3.214286,1.071429,3.214286
4,4.285714,3.214286,3.214286,1.071429,3.214286
...,...,...,...,...,...
13765,4.285714,3.214286,3.214286,1.071429,3.214286
13766,4.285714,3.214286,3.214286,1.071429,3.214286
13767,4.285714,3.214286,3.214286,1.071429,3.214286
13768,4.285714,3.214286,3.214286,1.071429,3.214286


In [16]:
Y_test

,GA,PSO,DE,CMAES,ES
56974,3.0,4.0,5.0,1.0,2.0
64937,5.0,1.0,3.0,2.0,4.0
35762,4.0,5.0,1.0,2.0,3.0
55640,5.0,2.0,4.0,1.0,3.0
32861,5.0,3.0,4.0,2.0,1.0
...,...,...,...,...,...
7969,5.0,1.5,1.5,3.0,4.0
48678,2.0,2.0,2.0,4.0,5.0
49828,5.0,3.0,4.0,1.0,2.0
22067,5.0,1.0,4.0,2.0,3.0


In [17]:
h=a

NameError: name 'a' is not defined

In [ ]:
Y_pred_with_meta = pd.concat([Y_pred.reset_index(drop=True), Y_test_meta.reset_index(drop=True)], axis=1)
Y_test_with_meta = pd.concat([Y_test.reset_index(drop=True), Y_test_meta.reset_index(drop=True)], axis=1)
h = Y_pred_with_meta.merge(Y_test_with_meta, on=['problem1', 'problem2', 'instance1', 'instance2', 'alpha', 'dim'], how='outer', suffixes=['_pred', '_test'])

In [ ]:
h

In [ ]:
def pairwise_errors(df1, df2, algorithms):
    s = []
    for al1 in algorithms:
        for al2 in algorithms:
            if al1 == al2:
                continue
            l = ((h[f'{al1}_pred']>h[f'{al2}_pred']) == (h[f'{al1}_test']>h[f'{al2}_test']))
            print(al1, al2, l.mean())
            s.append(l.to_list())
    s = np.array(s)
    print(s.shape)
    print()

pairwise_errors(v1, v2, algorithms)

In [ ]:
GA PSO 0.7282087282087282
GA DE 0.7707967707967708
GA CMAES 0.8642798642798643
GA ES 0.8990873990873991
PSO GA 0.7344097344097344
PSO DE 0.6282321282321283
PSO CMAES 0.8149058149058149
PSO ES 0.7632502632502632
DE GA 0.7802152802152802
DE PSO 0.6642681642681643
DE CMAES 0.8483678483678484
DE ES 0.8275418275418276
CMAES GA 0.8557973557973558
CMAES PSO 0.7843687843687843
CMAES DE 0.8172458172458172
CMAES ES 0.8098163098163098
ES GA 0.8985023985023985
ES PSO 0.7633087633087633
ES DE 0.8266643266643267
ES CMAES 0.8116883116883117
(20, 17094)

In [ ]:
GA PSO 0.7127062127062127
GA DE 0.7392067392067392
GA CMAES 0.7715572715572716
GA ES 0.8012753012753012
PSO GA 0.748040248040248
PSO DE 0.5736515736515736
PSO CMAES 0.6133731133731134
PSO ES 0.49023049023049026
DE GA 0.7745992745992746
DE PSO 0.5571545571545572
DE CMAES 0.603954603954604
DE ES 0.4234819234819235
CMAES GA 0.7855387855387855
CMAES PSO 0.655025155025155
CMAES DE 0.6438516438516438
CMAES ES 0.7492687492687493
ES GA 0.8052533052533053
ES PSO 0.5148590148590149
ES DE 0.5803205803205803
ES CMAES 0.7448227448227448
(20, 17094)

In [ ]:
#Y_pred_with_meta

In [ ]:
#Y_test_with_meta

In [ ]:
#v2.querry()

In [ ]:
#h = v1.merge(v2, on=['problem1', 'problem2', 'instance1', 'instance2', 'alpha', 'dim'], how='outer')
#h

In [ ]:
#((h['GA_x']>h['PSO_x']) == (h['GA_y']>h['PSO_y']))

In [ ]:
#(h['GA_x']>h['PSO_x'])

In [ ]:
#(h['GA_y']>h['PSO_y'])

In [ ]:
#h.groupby(['problem1', 'problem2', 'instance1', 'instance2', 'alpha', 'dim']).mean().reset_index()

In [ ]:
GA PSO 0.7282087282087282
GA DE 0.7707967707967708
GA CMAES 0.8642798642798643
GA ES 0.8990873990873991
PSO GA 0.7344097344097344
PSO DE 0.6282321282321283
PSO CMAES 0.8149058149058149
PSO ES 0.7632502632502632
DE GA 0.7802152802152802
DE PSO 0.6642681642681643
DE CMAES 0.8483678483678484
DE ES 0.8275418275418276
CMAES GA 0.8557973557973558
CMAES PSO 0.7843687843687843
CMAES DE 0.8172458172458172
CMAES ES 0.8098163098163098
ES GA 0.8985023985023985
ES PSO 0.7633087633087633
ES DE 0.8266643266643267
ES CMAES 0.8116883116883117

In [ ]:
Y_test

In [ ]:
Y_pred

In [ ]:
f = h

In [ ]:
mask = df['problem1'] > df['problem2']
#df.loc[mask, ['problem1', 'problem2']] = df.loc[mask, ['problem2', 'problem1']].copy()
maxv = df[['problem2', 'problem1']].max(axis=1)
minv = df[['problem2', 'problem1']].min(axis=1)
df['problem1'] = minv
df['problem2'] = maxv

df.loc[mask, 'alpha'] = 1 - df.loc[mask, 'alpha']

In [ ]:
df

In [ ]:
algos = ['GA', 'PSO', 'DE', 'CMAES', 'ES']
rdf = df[algos].rank(axis=1)
for column in df.columns.difference(algos):
    rdf[column]=df[column]

In [ ]:
pure = rdf.query('alpha==0.0').groupby('problem2').mean().reset_index()
pure

In [ ]:
fig, ax = plt.subplots(figsize=(9, 2))

im = ax.imshow(pure[algos].T, aspect='auto')
plt.xticks(range(24), range(1, 25))
plt.yticks(range(len(algos)), algos)

for (j,i),label in np.ndenumerate(pure[algos].T):
    ax.text(i,j, round(float(label), 1),ha='center',va='center')

plt.savefig(f'figures/bbob_ranks.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
rdf.alpha = rdf.alpha.round(2)

In [ ]:
lin = rdf.query('(problem2==20 and problem1 == 1)').groupby('alpha').mean().reset_index().sort_values('alpha', ascending=False)

In [ ]:
lin

In [ ]:
fig, ax = plt.subplots(figsize=(9, 2))

im = ax.imshow(lin[algos].T, aspect='auto')
plt.xticks(range(len(lin)), [round(x, 3) for x in lin.alpha])
plt.yticks(range(len(algos)), algos)

for (j,i),label in np.ndenumerate(lin[algos].T):
    ax.text(i,j, round(float(label), 1),ha='center',va='center')

plt.savefig(f'figures/combination.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
#!pip install matplotlib